## Compare extraction method in the same animal

This script allows you to compare point clouds that were extracted manually or automatically. Manually extracted point clouds can be found in dataset with [DOI: 10.25493/AJVE-B8G](https://search.kg.ebrains.eu/instances/27dd2588-f4da-406c-9d01-8569bf912b0e) and the automatically extracted point clouds can be found in dataset with [DOI: 10.25493/QT31-PJS](https://search.kg.ebrains.eu/instances/4660e79b-a731-40ac-905e-46d0d11c0dd5). The output file can be visualized in [Meshview](https://meshview.apps.hbp.eu/?atlas=ABA_Mouse_CCFv3_2017_25um). 

For this example we chose the following two cases:
- **114292355_SSPll** (manually extracted points are depicted in light blue, automatically extracted points are depicted in dark blue)
- **141602484_MOs** (manually extracted points are depicted in beige, automatically extracted points are depicted in brown)



In [1]:
# Import packages
import json
import os
from pathlib import Path


## Create output folder and find the experiment files
Ensure that the experiment files downloaded from EBRAINS are in the data folder

In [ ]:
# Data folder
data_folder = os.path.join("..", "data")

# Check if output folder exist, if not create the folder
output_folder = os.path.join("..", "output")
if not os.path.exists(output_folder):
   os.makedirs(output_folder)
   print("The new directory is created!")

## Define the cases of interest

In [2]:

data = {
    "case_names": [
        {"type": "manual", "name": "WT-control_114292355_SSPll"},   # manual extraction
        {"type": "manual", "name": "WT-control_141602484_MOs"},     # manual extraction
        {"type": "automatic", "name": "114292355_SSPll"},           # automatic extraction
        {"type": "automatic", "name": "141602484_MOs"}              # automatic extraction
    ],
    "colors": {
        "manual_114292355_SSPll" : [171, 219, 227], 
        "automatic_114292355_SSPll" : [6, 57, 112], 
        "manual_141602484_MOs" : [234, 182, 118], 
        "automatic_141602484_MOs" : [135, 62, 35]
    }
}


## Create the JSON file

In [3]:
# function to speed up writing the JSON files (works well for large point clouds)
def chunked_json_dump(data, file, chunk_size=1024):
    for i in range(0, len(data), chunk_size):
        file.write(json.dumps(data[i:i+chunk_size], ensure_ascii=False))


points_list = []

for exp in data["case_names"]:
    extraction_type = exp["type"]
    expName = exp["name"]
    fname = sorted(Path().rglob(f"{expName}.json"))[0]

    with open(fname, "r") as f:
        points = json.load(f)

    # select the correct area (automatic ones have 4 brain regions)
    point_cloud = points[0] if extraction_type == "manual" else points[2]

    # Update name and colors
    name = "_".join(expName.split("_")[-2:])
    point_cloud["name"] = f"{extraction_type}_{name}"
    rgb = data["colors"][f"{extraction_type}_{name}"]
    point_cloud.update({"r": rgb[0], "g": rgb[1], "b": rgb[2]})

    # append the point clouds to the list to create only one output file
    points_list.append(point_cloud)

# Saving the JSON file
out_file = os.path.join(output_folder,f"compare_extraction-method.json")
print(f"Saving file {out_file}")
with open(out_file, 'w', encoding='utf-8') as fi:
    chunked_json_dump(points_list, fi)

print("Done!")

Saving file output\compare_extraction-method.json
Done!
